In [2]:
import mlflow

import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression , Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope




Run following command in terminal

python -m mlflow server --backend-store-uri sqlite:///backend.db

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000") 
mlflow.set_experiment("Electricity Demand Prediction")

2024/08/19 10:06:18 INFO mlflow.tracking.fluent: Experiment with name 'Electricity Demand Prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1724061978581, experiment_id='1', last_update_time=1724061978581, lifecycle_stage='active', name='Electricity Demand Prediction', tags={}>

In [4]:
def read_dataframe(filename):
    
    """
    Reads a CSV file into a DataFrame, processes date and demand columns, and adds additional features.

    :param filename: Path to the CSV file to be read.
    :return: DataFrame with processed columns and additional features (year, month, day, hour, day_of_week, is_weekend, holiday).
    """
    
    df = pd.read_csv(filename)
    df['date'] = pd.to_datetime(df['date'])
    df['demand'] = pd.to_numeric(df['demand'], errors='coerce').astype('float')
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hr'] = df['date'].dt.hour
    df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6
    df['is_weekend'] = df['date'].dt.dayofweek >= 5  # True if weekend, False otherwise
    
    holidays = calendar().holidays(start=df['date'].min(), end=df['date'].max())
    df['holiday'] = df['date'].isin(holidays).astype(int)
    #display(df.head())
    #print(df.dtypes)
    
    
    return df

In [5]:
dataset_path = "dataset.csv"

In [6]:
df = read_dataframe(dataset_path)

In [7]:
# Define features and target
X = df.drop('demand', axis=1)
X = X.drop('date', axis=1)

y = df['demand']

In [8]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [10]:
def objective(params):
    """
    Trains an XGBoost model with specified parameters and logs metrics using MLflow.

    :param params: Dictionary of hyperparameters for the XGBoost model.
    :return: Dictionary containing the loss ('rmse') and the status of the optimization.
    """
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        mlflow.xgboost.autolog()
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [11]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)
print(best_result)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2024/08/19 10:06:22 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.



[0]	validation-rmse:2941.88622                        
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:06:22] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:2749.55909                        
[2]	validation-rmse:2573.78054                        
[3]	validation-rmse:2411.11635                        
[4]	validation-rmse:2262.64542                        
[5]	validation-rmse:2126.24182                        
[6]	validation-rmse:2002.47304                        
[7]	validation-rmse:1887.54471                        
[8]	validation-rmse:1783.07394                        
[9]	validation-rmse:1686.85896                        
[10]	validation-rmse:1596.57982                       
[11]	validation-rmse:1514.90318                       
[12]	validation-rmse:1441.69401                       
[13]	validation-rmse:1375.59182                       
[14]	validation-rmse:1316.19447                       
[15]	validation-rmse:1262.28575                       
[16]	validation-rmse:1213.37124                       
[17]	validation-rmse:1167.58315                       
[18]	validation-rmse:1126.97144                       
[19]	valid

...

[0]	validation-rmse:3002.22505                                                  
 98%|█████████▊| 49/50 [10:20<00:12, 12.02s/trial, best loss: 379.7598569833861]

/home/codespace/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:16:41] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:2861.81759                                                  
[2]	validation-rmse:2728.79858                                                  
[3]	validation-rmse:2603.32495                                                  
[4]	validation-rmse:2484.80838                                                  
[5]	validation-rmse:2372.92592                                                  
[6]	validation-rmse:2266.52623                                                  
[7]	validation-rmse:2166.38334                                                  
[8]	validation-rmse:2071.52050                                                  
[9]	validation-rmse:1982.05484                                                  
[10]	validation-rmse:1897.34845                                                 
[11]	validation-rmse:1818.49392                                                 
[12]	validation-rmse:1743.27698                                                 
[13]	validation-rmse:1672.07

2024/08/19 10:17:01 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/08/19 10:17:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:17:01] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/08/19 10:17:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/python/3.10.13/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils 

100%|██████████| 50/50 [10:43<00:00, 12.87s/trial, best loss: 379.7598569833861]
{'learning_rate': 0.18891478774394413, 'max_depth': 9.0, 'min_child_weight': 6.642316776962908, 'reg_alpha': 0.012540977368792963, 'reg_lambda': 0.012396118588604442}


In [11]:
new_observation = pd.DataFrame({
    'temperature' : [21],
    'year': [2024],
    'month': [8],
    'day': [12],
    'hr': [3],
    'day_of_week': [1],
    'is_weekend': [0],
    'holiday': [0]
})

In [12]:
import mlflow
logged_model = 'runs:/0ac14477b69842e195919d1cab69d066/model'

# Load model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(logged_model)



/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


array([20198.39], dtype=float32)

In [13]:
loaded_model.predict(pd.DataFrame(new_observation))

array([20198.39], dtype=float32)

In [13]:
print(best_result)

{'learning_rate': 0.18891478774394413, 'max_depth': 9.0, 'min_child_weight': 6.642316776962908, 'reg_alpha': 0.012540977368792963, 'reg_lambda': 0.012396118588604442}
